# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from census import Census

# Import API key
from api_keys import g_key

In [2]:
%matplotlib inline

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv("weather_data.csv")
weather_data = weather_data.drop(weather_data.columns[[0]], axis=1)
weather_data.head()

,City,Country,Date,Cloud Cover,Humidity,Maximum Temperature,Wind Speed,Latitude,Longitude
0,São Filipe,CV,1602817994,35,77,81.10,11.90,14.90,-24.50
1,Nome,US,1602817995,90,86,39.20,18.34,64.50,-165.41
2,Lorengau,PG,1602817997,100,71,84.07,4.90,-2.02,147.27
3,Cabo San Lucas,MX,1602817859,5,74,82.40,11.41,22.89,-109.91
4,Barrow,US,1602818001,75,79,19.40,14.99,71.29,-156.79


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)

In [13]:
locations = weather_data[["Latitude", "Longitude"]]
humidity = weather_data["Humidity"]
fig = gmaps.figure()
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100, 
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloud Cover"] == 0) & \
            (weather_data["Maximum Temperature"] >= 70) & (weather_data["Maximum Temperature"] <= 80)].dropna()
narrowed_df

,City,Country,Date,Cloud Cover,Humidity,Maximum Temperature,Wind Speed,Latitude,Longitude
41,Bayjī,IQ,1602818062,0,30,70.81,9.06,34.93,43.49
48,Bosaso,SO,1602818073,0,63,79.99,1.34,11.28,49.18
88,Khanbari,NP,1602818140,0,53,78.13,1.59,27.37,87.20
162,Amapá,BR,1602818264,0,91,71.78,2.64,1.00,-52.00
181,Grand Gaube,MU,1602818297,0,75,75.99,1.99,-20.01,57.66
194,Wagar,SD,1602818318,0,49,79.48,5.99,16.15,36.20
310,Breves,BR,1602818513,0,79,77.14,6.51,-1.68,-50.48
378,Aswān,EG,1602818631,0,29,78.85,9.33,24.09,32.91
500,Kita,ML,1602818836,0,72,76.60,1.95,13.03,-9.49


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_df.loc[:,["City","Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
41,Bayjī,IQ,34.93,43.49,
48,Bosaso,SO,11.28,49.18,
88,Khanbari,NP,27.37,87.20,
162,Amapá,BR,1.00,-52.00,
181,Grand Gaube,MU,-20.01,57.66,
194,Wagar,SD,16.15,36.20,
310,Breves,BR,-1.68,-50.48,
378,Aswān,EG,24.09,32.91,
500,Kita,ML,13.03,-9.49,


In [8]:
target_radius = 5000
target_type = "hotel"

params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key}

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [10]:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    city = row["City"]
    params["location"] = f"{lat},{lon}"
    
    print(f"Retrieving City and Hotel Information, Index No. {index}: {city}.")
    response = requests.get(url, params=params).json()
    results = response['results']

    try:
        print(f"{results[0]['name']} is the closest hotel in {city}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Hotel information not found, skipping to the next city...")  
    print('-'*30)
    time.sleep(1)

Retrieving City and Hotel Information, Index No. 41: Bayjī.
Baiji is the closest hotel in Bayjī.
------------------------------
Retrieving City and Hotel Information, Index No. 48: Bosaso.
Bosaso is the closest hotel in Bosaso.
------------------------------
Retrieving City and Hotel Information, Index No. 88: Khanbari.
Khandbari is the closest hotel in Khanbari.
------------------------------
Retrieving City and Hotel Information, Index No. 162: Amapá.
Teresinha is the closest hotel in Amapá.
------------------------------
Retrieving City and Hotel Information, Index No. 181: Grand Gaube.
Goodlands is the closest hotel in Grand Gaube.
------------------------------
Retrieving City and Hotel Information, Index No. 194: Wagar.
Wagga is the closest hotel in Wagar.
------------------------------
Retrieving City and Hotel Information, Index No. 310: Breves.
Breves is the closest hotel in Breves.
------------------------------
Retrieving City and Hotel Information, Index No. 378: Aswān.
Asw

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))